In [2]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import datetime
from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [3]:
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
## Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [6]:
TARGET = 'sales'         # Our main target
END_TRAIN = 1941         # Last day in train set, change this part for final
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [7]:
print('Load Main Data')
# Here are reafing all our data 
# without any limitations and dtype modification
train_df = pd.read_csv('sales_train_evaluation.csv') # change this part for final
prices_df = pd.read_csv('sell_prices.csv')
calendar_df = pd.read_csv('calendar.csv')

Load Main Data


In [8]:
# use cluster to remove some obs
cluster=pd.read_csv('cluster4.csv') # change this part for final

In [9]:
cluster.head()

,cluster,item_id,0,7,14,21,28,35,42,49,...,1876,1883,1890,1897,1904,1911,1918,1925,1932,1939
0,0,FOODS_1_004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.241116,1.857643,1.897480,1.285254,1.227945
1,0,FOODS_1_009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.037360,0.000000,0.000000,0.000000,0.000000,0.971359,2.727278,2.951438,2.876718,2.876718
2,0,FOODS_1_010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.422206,1.244430,2.666636,1.688870,1.911089,1.911089,1.466650,1.333318,1.377762,2.592563
3,0,FOODS_1_012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.203461,1.286679,1.325087,1.302682,1.277077,1.190658,1.465918,1.142648,1.091437,1.299482
4,0,FOODS_1_014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.760925,1.320694,1.700894,1.740915,1.660873,2.161136,1.720904,2.261188,2.221167,1.820957


In [10]:
cluster=cluster[['cluster', 'item_id']]

In [11]:
cluster['cluster'].value_counts()

2    1700
0     837
1     353
3     159
Name: cluster, dtype: int64

In [12]:
train_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [13]:
train_df=train_df.merge(cluster, on='item_id', how='left')

In [14]:
train_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,cluster
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,0,0,0,0,3,3,0,1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,0,0,0,0,0,2
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,0,2,3,0,1,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,4,0,1,3,0,2,6,2
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,2,1,0,0,2,1,0,2


In [15]:
train_df['cluster'].value_counts()

2    17000
0     8370
1     3530
3     1590
Name: cluster, dtype: int64

In [16]:
print('Create Grid')

# We can tranform horizontal representation 
# to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
# and labels are 'd_' coulmns

index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id','cluster']
grid_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

# If we look on train_df we see that 
# we don't have a lot of traning rows
# but each day can provide more train data
print('Train rows:', len(train_df), len(grid_df))

# To be able to make predictions
# we need to add "test set" to our grid
add_grid = pd.DataFrame()
for i in range(1,29):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)

# Remove some temoprary DFs
del temp_df, add_grid

# We will not need original train_df
# anymore and can remove it
del train_df

# You don't have to use df = df construction
# you can use inplace=True instead.
# like this
# grid_df.reset_index(drop=True, inplace=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# We can free some memory 
# by converting "strings" to categorical
# it will not affect merging and 
# we will not lose any valuable data
for col in ['id','item_id','dept_id','cat_id','store_id','state_id']:
    grid_df[col] = grid_df[col].astype('category')

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Create Grid
Train rows: 30490 59181090
    Original grid_df:   4.0GiB
     Reduced grid_df:   1.8GiB


In [17]:
grid_df.head() #1-1941:train

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,d_1,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,d_1,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,0.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,0.0


In [18]:
grid_df.tail() #1942-1969:test

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,d_1969,NaN
60034809,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,3,d_1969,NaN


In [19]:
grid_df.shape

(60034810, 9)

In [20]:
grid_df['day']=grid_df['d'].apply(lambda x:int(x.split('_')[1]))

In [21]:
grid_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,d_1,0.0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,0.0,1
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,d_1,0.0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,0.0,1
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,0.0,1


In [22]:
grid_df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN,1969
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN,1969
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN,1969
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,d_1969,NaN,1969
60034809,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,3,d_1969,NaN,1969


In [23]:
grid_df.dtypes

id          category
item_id     category
dept_id     category
cat_id      category
store_id    category
state_id    category
cluster        int64
d             object
sales        float64
day            int64
dtype: object

In [263]:
# remove cluster leading zero or few obs

In [24]:
grid_df['cluster'].value_counts()

2    33473000
0    16480530
1     6950570
3     3130710
Name: cluster, dtype: int64

In [25]:
grid_df=grid_df.drop(grid_df[(grid_df['cluster']==1)&(grid_df['day']<=500)].index)

In [26]:
grid_df.shape

(58269810, 10)

In [27]:
60034810-58269810

1765000

In [30]:
grid_df[grid_df['cluster']==1].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day
15245002,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,d_501,0.0,501
15245017,HOBBIES_1_018_CA_1_evaluation,HOBBIES_1_018,HOBBIES_1,HOBBIES,CA_1,CA,1,d_501,2.0,501
15245026,HOBBIES_1_027_CA_1_evaluation,HOBBIES_1_027,HOBBIES_1,HOBBIES,CA_1,CA,1,d_501,0.0,501
15245053,HOBBIES_1_054_CA_1_evaluation,HOBBIES_1_054,HOBBIES_1,HOBBIES,CA_1,CA,1,d_501,0.0,501
15245064,HOBBIES_1_066_CA_1_evaluation,HOBBIES_1_066,HOBBIES_1,HOBBIES,CA_1,CA,1,d_501,0.0,501


In [31]:
grid_df[grid_df['cluster']==0].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,d_1,0.0,1
5,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,d_1,0.0,1
6,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,d_1,0.0,1
10,HOBBIES_1_011_CA_1_evaluation,HOBBIES_1_011,HOBBIES_1,HOBBIES,CA_1,CA,0,d_1,0.0,1
12,HOBBIES_1_013_CA_1_evaluation,HOBBIES_1_013,HOBBIES_1,HOBBIES,CA_1,CA,0,d_1,0.0,1


In [32]:
grid_df=grid_df.drop(grid_df[(grid_df['cluster']==0)&(grid_df['day']<=250)].index)

In [33]:
grid_df.shape

(56177310, 10)

In [34]:
58269810-56177310

2092500

In [35]:
grid_df[grid_df['cluster']==0].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day
7622500,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251
7622505,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251
7622506,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251
7622510,HOBBIES_1_011_CA_1_evaluation,HOBBIES_1_011,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251
7622512,HOBBIES_1_013_CA_1_evaluation,HOBBIES_1_013,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251


In [36]:
grid_df[grid_df['cluster']==3].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day
44,HOBBIES_1_045_CA_1_evaluation,HOBBIES_1_045,HOBBIES_1,HOBBIES,CA_1,CA,3,d_1,0.0,1
45,HOBBIES_1_046_CA_1_evaluation,HOBBIES_1_046,HOBBIES_1,HOBBIES,CA_1,CA,3,d_1,0.0,1
120,HOBBIES_1_125_CA_1_evaluation,HOBBIES_1_125,HOBBIES_1,HOBBIES,CA_1,CA,3,d_1,0.0,1
153,HOBBIES_1_158_CA_1_evaluation,HOBBIES_1_158,HOBBIES_1,HOBBIES,CA_1,CA,3,d_1,0.0,1
180,HOBBIES_1_186_CA_1_evaluation,HOBBIES_1_186,HOBBIES_1,HOBBIES,CA_1,CA,3,d_1,0.0,1


In [37]:
grid_df=grid_df.drop(grid_df[(grid_df['cluster']==3)&(grid_df['day']<=1000)].index)

In [38]:
grid_df.shape

(54587310, 10)

In [39]:
56177310-54587310

1590000

In [40]:
grid_df[grid_df['cluster']==0].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day
7622500,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251
7622505,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251
7622506,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251
7622510,HOBBIES_1_011_CA_1_evaluation,HOBBIES_1_011,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251
7622512,HOBBIES_1_013_CA_1_evaluation,HOBBIES_1_013,HOBBIES_1,HOBBIES,CA_1,CA,0,d_251,0.0,251


In [41]:
grid_df.reset_index(drop=True, inplace=True)

In [42]:
grid_df.shape

(54587310, 10)

In [43]:
60034810-54587310

5447500

In [44]:
grid_df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day
54587305,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN,1969
54587306,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN,1969
54587307,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,2,d_1969,NaN,1969
54587308,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,d_1969,NaN,1969
54587309,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,3,d_1969,NaN,1969


In [45]:
print('Release week')

# It seems that leadings zero values
# in each train_df item row
# are not real 0 sales but mean
# absence for the item in the store
# we can safe some memory by removing
# such zeros

# Prices are set by week
# so it we will have not very accurate release week 
release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id','item_id','release']

Release week


In [46]:
release_df.head() # release: the first wm_yr_wk has price

,store_id,item_id,release
0,CA_1,FOODS_1_001,11101
1,CA_1,FOODS_1_002,11101
2,CA_1,FOODS_1_003,11101
3,CA_1,FOODS_1_004,11206
4,CA_1,FOODS_1_005,11101


In [47]:
# Now we can merge release_df
grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
del release_df

In [48]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day,release
888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1,3.0,1,11101
17888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_2,0.0,2,11101
34888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_3,0.0,3,11101
51888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_4,1.0,4,11101
68888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_5,4.0,5,11101


In [49]:
# We want to remove some "zeros" rows
# from grid_df 
# to do it we need wm_yr_wk column
# let's merge partly calendar_df to have it
grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk','d']], ['d'])

In [50]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day,release,wm_yr_wk
888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1,3.0,1,11101,11101
17888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_2,0.0,2,11101,11101
34888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_3,0.0,3,11101,11101
51888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_4,1.0,4,11101,11101
68888,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_5,4.0,5,11101,11101


In [52]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].tail()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day,release,wm_yr_wk
54436472,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1965,NaN,1965,11101,11620
54466962,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1966,NaN,1966,11101,11620
54497452,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1967,NaN,1967,11101,11620
54527942,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1968,NaN,1968,11101,11621
54558432,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1969,NaN,1969,11101,11621


In [53]:
grid_df.shape

(54587310, 12)

In [54]:
# Now we can cutoff some rows 
# and safe memory 
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']]
grid_df = grid_df.reset_index(drop=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

    Original grid_df:   2.5GiB


In [55]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,day,release,wm_yr_wk
574,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1,3.0,1,11101,11101
11321,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_2,0.0,2,11101,11101
22068,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_3,0.0,3,11101,11101
32815,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_4,1.0,4,11101,11101
43562,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_5,4.0,5,11101,11101


In [56]:
grid_df.shape  

(47617813, 12)

In [57]:
54587310-47617813

6969497

In [58]:
grid_df.drop('day', axis=1, inplace=True)

In [59]:
grid_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,release,wm_yr_wk
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,12.0,11101,11101
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,2.0,11101,11101
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,0.0,11101,11101
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,0.0,11101,11101
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,2,d_1,4.0,11101,11101


In [60]:
# Should we keep release week 
# as one of the features?
# Only good CV can give the answer.
# Let's minify the release values.
# Min transformation will not help here 
# as int16 -> Integer (-32768 to 32767)
# and our grid_df['release'].max() serves for int16
# but we have have an idea how to transform 
# other columns in case we will need it
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

     Reduced grid_df:   1.9GiB


In [61]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].head()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,release,wm_yr_wk
574,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1,3.0,0,11101
11321,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_2,0.0,0,11101
22068,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_3,0.0,0,11101
32815,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_4,1.0,0,11101
43562,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_5,4.0,0,11101


In [62]:
grid_df[(grid_df['store_id']=='CA_1') & (grid_df['item_id']=='FOODS_1_001')].tail()

,id,item_id,dept_id,cat_id,store_id,state_id,cluster,d,sales,release,wm_yr_wk
47466975,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1965,NaN,0,11620
47497465,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1966,NaN,0,11620
47527955,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1967,NaN,0,11620
47558445,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1968,NaN,0,11621
47588935,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,d_1969,NaN,0,11621


In [63]:
print('Save Part 1')

# We have our BASE grid ready
# and can save it as pickle file
# for future use (model training)
grid_df.to_pickle('grid_part_1.pkl')

print('Size:', grid_df.shape)

Save Part 1
Size: (47617813, 11)


In [64]:
# add price feature
prices_df.head(5)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [65]:
print('Prices')

# We can do some basic aggregations
prices_df['price_max'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
prices_df['price_min'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
prices_df['price_std'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
prices_df['price_mean'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# and do price normalization (min/max scaling)
prices_df['price_norm'] = prices_df['sell_price']/prices_df['price_max']

Prices


# #price_rank_dept

In [66]:
# price_rank_dept
prices_df["dept"]=prices_df["item_id"].apply(lambda x: x.split('_'))
prices_df["dept"]=prices_df["dept"].apply(lambda x: str(x[0:2]))

cat=prices_df["dept"].value_counts().index

a=pd.DataFrame()
k=10
for i in range(len(cat))    :
    temp=pd.cut(prices_df[prices_df["dept"]==cat[i]]['sell_price'],k,labels = range(k))
    a=pd.concat([a,temp])
a.columns=["price_rank_dept"]
a.sort_index(inplace=True)

prices_df=pd.concat([prices_df,a],axis=1)
prices_df["price_rank_dept"]=prices_df["price_rank_dept"].astype("category")
prices_df.drop(["dept"],axis=1,inplace=True)
del a,cat,k
gc.collect()

42

In [67]:
prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_rank_dept
0,CA_1,HOBBIES_1_001,11325,9.58,9.58,8.26,0.152139,8.285714,1.000000,3
1,CA_1,HOBBIES_1_001,11326,9.58,9.58,8.26,0.152139,8.285714,1.000000,3
2,CA_1,HOBBIES_1_001,11327,8.26,9.58,8.26,0.152139,8.285714,0.862213,2
3,CA_1,HOBBIES_1_001,11328,8.26,9.58,8.26,0.152139,8.285714,0.862213,2
4,CA_1,HOBBIES_1_001,11329,8.26,9.58,8.26,0.152139,8.285714,0.862213,2


In [68]:
# Some items are can be inflation dependent
# and some items are very "stable"
# check how many unique values each group has
prices_df['price_nunique'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
prices_df['item_nunique'] = prices_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

In [69]:
prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_rank_dept,price_nunique,item_nunique
0,CA_1,HOBBIES_1_001,11325,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,3.0,3
1,CA_1,HOBBIES_1_001,11326,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,3.0,3
2,CA_1,HOBBIES_1_001,11327,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
3,CA_1,HOBBIES_1_001,11328,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5
4,CA_1,HOBBIES_1_001,11329,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5


In [70]:
# I would like some "rolling" aggregations
# but would like months and years as "window"
calendar_prices = calendar_df[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_prices

# Now we can add price "momentum" (some sort of)
# Shifted by week 
# by month mean
# by year mean
prices_df['price_momentum'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_m'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

# weather-box

In [71]:
## https://zh.weatherspark.com/y/
# 'temperature_high' 
# 'temperature_low' 
# temperature: 
# 极冷 0
# 冰冻 1
# 很冷 2
# 寒冷 3
# 凉爽 4
# 舒适 5
# 暖和 6
# 热 7
# 热 8

In [72]:
ca_mask=(prices_df["store_id"]=="CA_1")|(prices_df["store_id"]=="CA_2")|(prices_df["store_id"]=="CA_3")|(prices_df["store_id"]=="CA_4")
tx_mask=(prices_df["store_id"]=="TX_1")|(prices_df["store_id"]=="TX_2")|(prices_df["store_id"]=="TX_3")
wi_mask=(prices_df["store_id"]=="WI_1")|(prices_df["store_id"]=="WI_2")|(prices_df["store_id"]=="WI_3")

In [73]:
#WEATHER PARMS
prices_df['temperature_high']=np.nan
prices_df['temperature_con']=np.nan
prices_df['rainfall_m']=np.nan
prices_df['snow_m']=np.nan

## wi_mask 
# temperature_con

prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==1))].index,'temperature_con']=1
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==2))].index,'temperature_con']=2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==3))].index,'temperature_con']=2.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==4))].index,'temperature_con']=3.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==5))].index,'temperature_con']=4.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==6))].index,'temperature_con']=5.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==7))].index,'temperature_con']=6
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==8))].index,'temperature_con']=6
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==9))].index,'temperature_con']=5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==10))].index,'temperature_con']=4
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==11))].index,'temperature_con']=3
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==12))].index,'temperature_con']=2

# temperature_high

prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==1))].index,'temperature_high']=0.5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==2))].index,'temperature_high']=2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==3))].index,'temperature_high']=6
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==4))].index,'temperature_high']=12
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==5))].index,'temperature_high']=18
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==6))].index,'temperature_high']=(21+27)/2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==7))].index,'temperature_high']=27
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==8))].index,'temperature_high']=25
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==9))].index,'temperature_high']=22
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==10))].index,'temperature_high']=(17+12)/2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==11))].index,'temperature_high']=(12+4)/2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==12))].index,'temperature_high']=(4+0)/2

# rainfall_m

prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==1))].index,'rainfall_m']=16
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==2))].index,'rainfall_m']=21
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==3))].index,'rainfall_m']=42
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==4))].index,'rainfall_m']=72
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==5))].index,'rainfall_m']=80
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==6))].index,'rainfall_m']=86
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==7))].index,'rainfall_m']=80
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==8))].index,'rainfall_m']=82
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==9))].index,'rainfall_m']=76
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==10))].index,'rainfall_m']=63
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==11))].index,'rainfall_m']=50
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==12))].index,'rainfall_m']=31

# snow_m

prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==1))].index,'snow_m']=11
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==2))].index,'snow_m']=10
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==3))].index,'snow_m']=5
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==4))].index,'snow_m']=2
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==5))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==6))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==7))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==8))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==9))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==10))].index,'snow_m']=0
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==11))].index,'snow_m']=3
prices_df.loc[prices_df[(wi_mask&(prices_df["month"]==12))].index,'snow_m']=9


In [74]:
## ca_mask 
# temperature_con

prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==1))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==2))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==3))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==4))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==5))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==6))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==7))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==8))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==9))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==10))].index,'temperature_con']=6
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==11))].index,'temperature_con']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==12))].index,'temperature_con']=5

# temperature_high

prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==1))].index,'temperature_high']=20
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==2))].index,'temperature_high']=20
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==3))].index,'temperature_high']=21
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==4))].index,'temperature_high']=22.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==5))].index,'temperature_high']=23.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==6))].index,'temperature_high']=25.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==7))].index,'temperature_high']=28
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==8))].index,'temperature_high']=29.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==9))].index,'temperature_high']=28.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==10))].index,'temperature_high']=25.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==11))].index,'temperature_high']=22
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==12))].index,'temperature_high']=20

# rainfall_m

prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==1))].index,'rainfall_m']=69
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==2))].index,'rainfall_m']=79
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==3))].index,'rainfall_m']=50
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==4))].index,'rainfall_m']=20
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==5))].index,'rainfall_m']=5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==6))].index,'rainfall_m']=1.5
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==7))].index,'rainfall_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==8))].index,'rainfall_m']=1
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==9))].index,'rainfall_m']=4
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==10))].index,'rainfall_m']=12
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==11))].index,'rainfall_m']=29
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==12))].index,'rainfall_m']=52

# snow_m

prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==1))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==2))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==3))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==4))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==5))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==6))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==7))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==8))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==9))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==10))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==11))].index,'snow_m']=0
prices_df.loc[prices_df[(ca_mask&(prices_df["month"]==12))].index,'snow_m']=0

In [75]:
## tx_mask 
# temperature_con(气候概要)

prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==1))].index,'temperature_con']=4
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==2))].index,'temperature_con']=4
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==3))].index,'temperature_con']=5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==4))].index,'temperature_con']=6
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==5))].index,'temperature_con']=6
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==6))].index,'temperature_con']=7
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==7))].index,'temperature_con']=8
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==8))].index,'temperature_con']=8
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==9))].index,'temperature_con']=7
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==10))].index,'temperature_con']=6
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==11))].index,'temperature_con']=5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==12))].index,'temperature_con']=4

# temperature_high

prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==1))].index,'temperature_high']=14
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==2))].index,'temperature_high']=17
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==3))].index,'temperature_high']=21
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==4))].index,'temperature_high']=24.5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==5))].index,'temperature_high']=28.5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==6))].index,'temperature_high']=32.5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==7))].index,'temperature_high']=35
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==8))].index,'temperature_high']=35.5
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==9))].index,'temperature_high']=31
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==10))].index,'temperature_high']=26
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==11))].index,'temperature_high']=19
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==12))].index,'temperature_high']=15

# rainfall_m

prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==1))].index,'rainfall_m']=50
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==2))].index,'rainfall_m']=59
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==3))].index,'rainfall_m']=70
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==4))].index,'rainfall_m']=85
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==5))].index,'rainfall_m']=102
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==6))].index,'rainfall_m']=80
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==7))].index,'rainfall_m']=42
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==8))].index,'rainfall_m']=47
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==9))].index,'rainfall_m']=64
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==10))].index,'rainfall_m']=89
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==11))].index,'rainfall_m']=70
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==12))].index,'rainfall_m']=62

# snow_m

prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==1))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==2))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==3))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==4))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==5))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==6))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==7))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==8))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==9))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==10))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==11))].index,'snow_m']=0
prices_df.loc[prices_df[(tx_mask&(prices_df["month"]==12))].index,'snow_m']=0

In [76]:
prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_rank_dept,...,item_nunique,month,year,price_momentum,price_momentum_m,price_momentum_y,temperature_high,temperature_con,rainfall_m,snow_m
0,CA_1,HOBBIES_1_001,11325,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,...,3,7,2013,NaN,1.127059,1.145166,28.0,6.0,0.0,0.0
1,CA_1,HOBBIES_1_001,11326,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,...,3,7,2013,1.000000,1.127059,1.145166,28.0,6.0,0.0,0.0
2,CA_1,HOBBIES_1_001,11327,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,...,5,7,2013,0.862213,0.971765,0.987377,28.0,6.0,0.0,0.0
3,CA_1,HOBBIES_1_001,11328,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,...,5,8,2013,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0
4,CA_1,HOBBIES_1_001,11329,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,...,5,8,2013,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0


In [77]:
del prices_df['month'], prices_df['year']

In [78]:
prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_rank_dept,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,temperature_high,temperature_con,rainfall_m,snow_m
0,CA_1,HOBBIES_1_001,11325,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,3.0,3,NaN,1.127059,1.145166,28.0,6.0,0.0,0.0
1,CA_1,HOBBIES_1_001,11326,9.58,9.58,8.26,0.152139,8.285714,1.000000,3,3.0,3,1.000000,1.127059,1.145166,28.0,6.0,0.0,0.0
2,CA_1,HOBBIES_1_001,11327,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,0.862213,0.971765,0.987377,28.0,6.0,0.0,0.0
3,CA_1,HOBBIES_1_001,11328,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0
4,CA_1,HOBBIES_1_001,11329,8.26,9.58,8.26,0.152139,8.285714,0.862213,2,3.0,5,1.000000,1.000000,0.987377,29.5,6.0,1.0,0.0


In [79]:
print('Merge prices and save part 2')

# Merge Prices
original_columns = list(grid_df)
grid_df = grid_df.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')
keep_columns = [col for col in list(grid_df) if col not in original_columns]
grid_df = grid_df[MAIN_INDEX+keep_columns]
grid_df = reduce_mem_usage(grid_df)

# Safe part 2
grid_df.to_pickle('grid_part_2.pkl')
print('Size:', grid_df.shape)

# We don't need prices_df anymore
del prices_df
gc.collect()

Merge prices and save part 2
Mem. usage decreased to 2226.67 Mb (64.7% reduction)
Size: (47617813, 18)


0

In [80]:
grid_df.head()

,id,d,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_rank_dept,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,temperature_high,temperature_con,rainfall_m,snow_m
0,HOBBIES_1_008_CA_1_evaluation,d_1,0.459961,0.500000,0.419922,0.019760,0.476318,0.919922,0,4.0,16,NaN,0.968750,0.949219,20.0,5.0,69.0,0.0
1,HOBBIES_1_009_CA_1_evaluation,d_1,1.559570,1.769531,1.559570,0.032745,1.764648,0.881348,0,2.0,9,NaN,0.885742,0.896484,20.0,5.0,69.0,0.0
2,HOBBIES_1_010_CA_1_evaluation,d_1,3.169922,3.169922,2.970703,0.046356,2.980469,1.000000,1,2.0,20,NaN,1.064453,1.043945,20.0,5.0,69.0,0.0
3,HOBBIES_1_012_CA_1_evaluation,d_1,5.980469,6.519531,5.980469,0.115967,6.468750,0.916992,1,3.0,71,NaN,0.921875,0.958984,20.0,5.0,69.0,0.0
4,HOBBIES_1_015_CA_1_evaluation,d_1,0.700195,0.720215,0.680176,0.011337,0.706543,0.972168,0,3.0,16,NaN,0.990234,1.001953,20.0,5.0,69.0,0.0


# calender feature

In [81]:
# add calender feature
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [82]:
calendar_df["event_name_1"].unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [83]:
calendar_df["event_type_1"].unique()

array([nan, 'Sporting', 'Cultural', 'National', 'Religious'], dtype=object)

In [84]:
## 上下半月
calendar_df['date']=pd.to_datetime(calendar_df['date'],format="%Y-%m-%d")
# calendar_df['is_first_half_month']=
calendar_df['is_first_half_month']=[ 1 if x.day>=15 else 0  for x in calendar_df['date']]
calendar_df['is_first_half_month']=calendar_df['is_first_half_month'].astype("category")

In [85]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_first_half_month
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,1
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0


周级别处理(wday ==1,2 是weekedn)

In [86]:
# calendar_df=calendar_df.reset_index()
calendar_df["weekend"]=[ 1 if x<=2 else 0  for x in calendar_df['wday']]

In [87]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_first_half_month,weekend
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,1,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,1,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0,0


# 节日前后处理 event_index

In [88]:
calendar_df["event_attention_after"]=np.nan
event_index=calendar_df[~calendar_df["event_name_1"].isnull()].index
for i in range(0,8):
    if event_index[-1]+i>=calendar_df.shape[0]:
        event_index=event_index[event_index<(event_index[-1]-i)]
    calendar_df.loc[event_index+i,"event_attention_after"]=7-i
calendar_df.loc[event_index,"event_attention_after"]=7
event_index=calendar_df[~calendar_df["event_name_1"].isnull()].index

In [89]:
calendar_df.head(100)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_first_half_month,weekend,event_attention_after
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1,1,NaN
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,1,1,NaN
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,1,0,NaN
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0,0,NaN
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0,0,NaN
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1,0,0,NaN
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0,0,0,NaN
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1,0,1,NaN
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1,0,1,7.0
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0,0,0,6.0


In [90]:
calendar_df["event_attention_bef"]=np.nan
for i in range(0,8):
    calendar_df.loc[calendar_df[~calendar_df["event_name_1"].isnull()].index-i,"event_attention_bef"]=i*-1
calendar_df.loc[calendar_df[~calendar_df["event_name_1"].isnull()].index,"event_attention_bef"]=0

In [91]:
calendar_df["event_attention_sum"]=calendar_df["event_attention_bef"]+calendar_df["event_attention_after"]

In [92]:
calendar_df["event_bef_weekend"]=np.nan
event_round_index=calendar_df[~calendar_df["event_attention_bef"].isnull()]
calendar_df.loc[event_round_index[event_round_index.wday<=2].index,"event_bef_weekend"]=1
calendar_df.loc[event_index,"event_bef_weekend"]=0

In [93]:
calendar_df["event_after_weekend"]=np.nan
event_round_index=calendar_df[~calendar_df["event_attention_after"].isnull()]
calendar_df.loc[event_round_index[event_round_index.wday<=2].index,"event_after_weekend"]=1
calendar_df.loc[event_index,"event_after_weekend"]=0

In [94]:
calendar_df.head(100)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,snap_CA,snap_TX,snap_WI,is_first_half_month,weekend,event_attention_after,event_attention_bef,event_attention_sum,event_bef_weekend,event_after_weekend
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,...,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,...,0,0,0,1,1,NaN,-7.0,NaN,1.0,NaN
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,...,0,0,0,1,0,NaN,-6.0,NaN,NaN,NaN
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,...,1,1,0,0,0,NaN,-5.0,NaN,NaN,NaN
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,...,1,0,1,0,0,NaN,-4.0,NaN,NaN,NaN
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,...,1,1,1,0,0,NaN,-3.0,NaN,NaN,NaN
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,...,1,0,0,0,0,NaN,-2.0,NaN,NaN,NaN
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,...,1,1,1,0,1,NaN,-1.0,NaN,1.0,NaN
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,...,1,1,1,0,1,7.0,0.0,7.0,0.0,0.0
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,...,1,1,0,0,0,6.0,-7.0,-1.0,NaN,NaN


In [95]:
calendar_df.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,snap_CA,snap_TX,snap_WI,is_first_half_month,weekend,event_attention_after,event_attention_bef,event_attention_sum,event_bef_weekend,event_after_weekend
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,...,0,1,1,1,0,NaN,-4.0,NaN,NaN,NaN
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,...,0,0,0,1,0,NaN,-3.0,NaN,NaN,NaN
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,...,0,0,0,1,0,NaN,-2.0,NaN,NaN,NaN
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,...,0,0,0,1,1,NaN,-1.0,NaN,1.0,NaN
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,...,0,0,0,1,1,7.0,0.0,7.0,0.0,0.0


#  NBA

In [96]:
# 赛事数据
calendar_df["NBA"]=np.nan
calendar_df["date"]=pd.to_datetime(calendar_df["date"])
#http://www.uhchina.com/lanqiu/2013-2014nba/
# http://sports.sina.com.cn/nba/playoff1213.html
a=[[2011,5,31],[2011,6,2],[2011,6,5],[2011,6,7],[2011,6,9],[2011,6,12],
   [2012,6,12],[2012,6,14],[2012,6,17],[2012,6,19],[2012,6,21],
   [2013,6,6],[2013,6,9],[2013,6,11],[2013,6,13],[2013,6,16],[2013,6,18],[2013,6,20],
   [2014,6,5],[2014,6,8],[2014,6,10],[2014,6,12],[2014,6,15], 
   [2015,6,4],[2015,6,7],[2015,6,9],[2015,6,11],[2015,6,14], [2015,6,16],
   [2016,6,2],[2016,6,5],[2016,6,8],[2016,6,10],[2016,6,13],[2016,6,16] ,[2016,6,19]
  ]
for i in range(0,len(a)):
    calendar_df.loc[calendar_df[calendar_df["date"]==datetime.datetime(*a[i])].index,"NBA"]=1

In [97]:
calendar_df[calendar_df["NBA"]==1]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,snap_TX,snap_WI,is_first_half_month,weekend,event_attention_after,event_attention_bef,event_attention_sum,event_bef_weekend,event_after_weekend,NBA
122,2011-05-31,11118,Tuesday,4,5,2011,d_123,NBAFinalsStart,Sporting,NaN,...,0,0,1,0,7.0,0.0,7.0,0.0,0.0,1.0
124,2011-06-02,11118,Thursday,6,6,2011,d_125,NaN,NaN,NaN,...,0,1,0,0,4.0,NaN,NaN,NaN,NaN,1.0
127,2011-06-05,11119,Sunday,2,6,2011,d_128,NaN,NaN,NaN,...,1,1,0,1,1.0,-7.0,-6.0,1.0,1.0,1.0
129,2011-06-07,11119,Tuesday,4,6,2011,d_130,NaN,NaN,NaN,...,1,0,0,0,0.0,-5.0,-5.0,NaN,NaN,1.0
131,2011-06-09,11119,Thursday,6,6,2011,d_132,NaN,NaN,NaN,...,1,1,0,0,NaN,-3.0,NaN,NaN,NaN,1.0
134,2011-06-12,11120,Sunday,2,6,2011,d_135,NBAFinalsEnd,Sporting,NaN,...,1,1,0,1,7.0,0.0,7.0,0.0,0.0,1.0
500,2012-06-12,11220,Tuesday,4,6,2012,d_501,NBAFinalsStart,Sporting,NaN,...,1,1,0,0,7.0,0.0,7.0,0.0,0.0,1.0
502,2012-06-14,11220,Thursday,6,6,2012,d_503,NaN,NaN,NaN,...,0,1,0,0,5.0,-7.0,-2.0,NaN,NaN,1.0
505,2012-06-17,11221,Sunday,2,6,2012,d_506,Father's day,Cultural,NaN,...,0,0,1,1,7.0,0.0,7.0,0.0,0.0,1.0
507,2012-06-19,11221,Tuesday,4,6,2012,d_508,NaN,NaN,NaN,...,0,0,1,0,0.0,-2.0,-2.0,NaN,NaN,1.0


In [98]:
grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
icols = ['date',
         'd',
         #category
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI', 
         'is_first_half_month', 
         'event_bef_weekend', 
         'event_after_weekend',
         'NBA',
         
         
         # numerical
        'event_attention_after',
         'event_attention_bef',
         'event_attention_sum',
         


        ]
grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

# Minify data
# 'snap_' columns we can convert to bool or int8
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI', 
         'is_first_half_month', 
         'event_bef_weekend', 
         'event_after_weekend',
         'NBA']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

grid_df['event_attention_after'] = grid_df['event_attention_after'].astype(np.float16) #v2
grid_df['event_attention_bef'] = grid_df['event_attention_bef'].astype(np.float16) #v2 
grid_df['event_attention_sum'] = grid_df['event_attention_sum'].astype(np.float16) #v2 
    
    
    
# Convert to DateTime
grid_df['date'] = pd.to_datetime(grid_df['date'])

# Make some features from date
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8) #day of month
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8) # week of year
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8) # month of year
grid_df['tm_q'] = grid_df['date'].dt.quarter.astype(np.int8) # quarter of year
grid_df['tm_y'] = grid_df['date'].dt.year # year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8) # minus min(year)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8) # week of month

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8) # day of week: monday start from 0, so saturday. is 5
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8) # weekend and weekday

# Remove date
del grid_df['date']

In [99]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47617813 entries, 0 to 47617812
Data columns (total 24 columns):
id                       category
d                        object
event_name_1             category
event_type_1             category
event_name_2             category
event_type_2             category
snap_CA                  category
snap_TX                  category
snap_WI                  category
is_first_half_month      category
event_bef_weekend        category
event_after_weekend      category
NBA                      category
event_attention_after    float16
event_attention_bef      float16
event_attention_sum      float16
tm_d                     int8
tm_w                     int8
tm_m                     int8
tm_q                     int8
tm_y                     int8
tm_wm                    int8
tm_dw                    int8
tm_w_end                 int8
dtypes: category(12), float16(3), int8(8), object(1)
memory usage: 1.9+ GB


In [100]:
grid_df.head()

,id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_first_half_month,...,event_attention_bef,event_attention_sum,tm_d,tm_w,tm_m,tm_q,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_evaluation,d_1,NaN,NaN,NaN,NaN,0,0,0,1,...,NaN,NaN,29,4,1,1,0,5,5,1
1,HOBBIES_1_009_CA_1_evaluation,d_1,NaN,NaN,NaN,NaN,0,0,0,1,...,NaN,NaN,29,4,1,1,0,5,5,1
2,HOBBIES_1_010_CA_1_evaluation,d_1,NaN,NaN,NaN,NaN,0,0,0,1,...,NaN,NaN,29,4,1,1,0,5,5,1
3,HOBBIES_1_012_CA_1_evaluation,d_1,NaN,NaN,NaN,NaN,0,0,0,1,...,NaN,NaN,29,4,1,1,0,5,5,1
4,HOBBIES_1_015_CA_1_evaluation,d_1,NaN,NaN,NaN,NaN,0,0,0,1,...,NaN,NaN,29,4,1,1,0,5,5,1


In [101]:
print('Save part 3')

# Safe part 3
grid_df.to_pickle('grid_part_3.pkl')
print('Size:', grid_df.shape)

# We don't need calendar_df anymore
del calendar_df
del grid_df

Save part 3
Size: (47617813, 24)


In [102]:
## Part 1
# Convert 'd' to int
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# Remove 'wm_yr_wk'
del grid_df['wm_yr_wk']
print(grid_df.info())
grid_df.to_pickle('grid_part_1.pkl')

del grid_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47617813 entries, 0 to 47617812
Data columns (total 10 columns):
id          category
item_id     category
dept_id     category
cat_id      category
store_id    category
state_id    category
cluster     int64
d           int16
sales       float64
release     int16
dtypes: category(6), float64(1), int16(2), int64(1)
memory usage: 1.2 GB
None


In [103]:
# group aggregation
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df[TARGET][grid_df['d']>1913] = np.nan # change this part for final

base_cols = list(grid_df)

print(base_cols)




icols =  [
            ['state_id'],
            ['store_id'],
            ['cat_id'],
            ['dept_id'],
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id'],
    
            ]

for col in icols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    grid_df['enc'+col_name+'mean'] = grid_df.groupby(col)[TARGET].transform('mean').astype(np.float16)
    grid_df['enc'+col_name+'std'] = grid_df.groupby(col)[TARGET].transform('std').astype(np.float16)

    
keep_cols = [col for col in list(grid_df) if col not in base_cols]
grid_df = grid_df[['id','d']+keep_cols]

['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'cluster', 'd', 'sales', 'release']
Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']


In [104]:
grid_df.head()

,id,d,enc_state_id_mean,enc_state_id_std,enc_store_id_mean,enc_store_id_std,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,...,enc_store_id_cat_id_mean,enc_store_id_cat_id_std,enc_store_id_dept_id_mean,enc_store_id_dept_id_std,enc_item_id_mean,enc_item_id_std,enc_item_id_state_id_mean,enc_item_id_state_id_std,enc_item_id_store_id_mean,enc_item_id_store_id_std
0,HOBBIES_1_008_CA_1_evaluation,1,1.575195,4.59375,1.637695,4.464844,0.709473,2.257812,0.866699,2.542969,...,1.004883,3.123047,1.261719,3.542969,4.687500,7.160156,6.582031,8.765625,7.230469,9.117188
1,HOBBIES_1_009_CA_1_evaluation,1,1.575195,4.59375,1.637695,4.464844,0.709473,2.257812,0.866699,2.542969,...,1.004883,3.123047,1.261719,3.542969,0.850098,1.752930,1.135742,2.099609,1.186523,2.015625
2,HOBBIES_1_010_CA_1_evaluation,1,1.575195,4.59375,1.637695,4.464844,0.709473,2.257812,0.866699,2.542969,...,1.004883,3.123047,1.261719,3.542969,0.610840,0.861816,0.562012,0.827148,0.719238,0.921875
3,HOBBIES_1_012_CA_1_evaluation,1,1.575195,4.59375,1.637695,4.464844,0.709473,2.257812,0.866699,2.542969,...,1.004883,3.123047,1.261719,3.542969,0.383057,0.690430,0.425781,0.726074,0.394043,0.649414
4,HOBBIES_1_015_CA_1_evaluation,1,1.575195,4.59375,1.637695,4.464844,0.709473,2.257812,0.866699,2.542969,...,1.004883,3.123047,1.261719,3.542969,4.425781,6.679688,6.921875,8.359375,6.066406,7.351562


In [105]:
print('Save Mean/Std encoding')
grid_df.to_pickle('mean_encoding_df.pkl')

Save Mean/Std encoding


In [125]:
# new lag data

In [106]:
train_df = pd.read_csv('sales_train_evaluation.csv') # change this part for final

In [107]:
print('Create Grid')

# We can tranform horizontal representation 
# to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
# and labels are 'd_' coulmns

index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

# If we look on train_df we see that 
# we don't have a lot of traning rows
# but each day can provide more train data
print('Train rows:', len(train_df), len(grid_df))

# To be able to make predictions
# we need to add "test set" to our grid
add_grid = pd.DataFrame()
for i in range(1,29):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)

# Remove some temoprary DFs
del temp_df, add_grid

# We will not need original train_df
# anymore and can remove it
del train_df

# You don't have to use df = df construction
# you can use inplace=True instead.
# like this
# grid_df.reset_index(drop=True, inplace=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# We can free some memory 
# by converting "strings" to categorical
# it will not affect merging and 
# we will not lose any valuable data
for col in ['id','item_id','dept_id','cat_id','store_id','state_id']:
    grid_df[col] = grid_df[col].astype('category')

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Create Grid
Train rows: 30490 59181090
    Original grid_df:   3.6GiB
     Reduced grid_df:   1.3GiB


In [108]:
grid_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0


In [109]:
grid_df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,NaN
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,NaN
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,NaN
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,NaN
60034809,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1969,NaN


In [110]:
# We need only 'id','d','sales'
# to make lags and rollings
grid_df = grid_df[['id','d','sales']]
SHIFT_DAY = 28
# Lags
# with 28 day shift
start_time = time.time()
print('Create lags')

LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)]
grid_df = grid_df.assign(**{
        '{}_lag_{}'.format(col, l): grid_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in LAG_DAYS
        for col in [TARGET]
    })

# Minify lag columns
for col in list(grid_df):
    if 'lag' in col:
        grid_df[col] = grid_df[col].astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

# Rollings
# with 28 day shift
start_time = time.time()
print('Create rolling aggs')

for i in [7,14,28,56,168]:
    print('Rolling period:', i)
    grid_df['rolling_mean_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
    grid_df['rolling_std_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

# Rollings
# with sliding shift
for d_shift in [1,7,14]: 
    print('Shifting period:', d_shift)
    for d_window in [7,14,28,56]:
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        grid_df[col_name] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
    
    
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

Create lags
6.99 min: Lags
Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 28
Rolling period: 56
Rolling period: 168
Shifting period: 1
Shifting period: 7
Shifting period: 14
13.47 min: Lags


In [112]:
grid_df.head()

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,...,rolling_mean_tmp_1_28,rolling_mean_tmp_1_56,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_28,rolling_mean_tmp_7_56,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_28,rolling_mean_tmp_14_56
0,HOBBIES_1_001_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
grid_df.tail()

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,...,rolling_mean_tmp_1_28,rolling_mean_tmp_1_56,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_28,rolling_mean_tmp_7_56,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_28,rolling_mean_tmp_14_56
60034805,FOODS_3_823_WI_3_evaluation,d_1969,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60034806,FOODS_3_824_WI_3_evaluation,d_1969,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60034807,FOODS_3_825_WI_3_evaluation,d_1969,NaN,2.0,0.0,1.0,0.0,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60034808,FOODS_3_826_WI_3_evaluation,d_1969,NaN,0.0,1.0,1.0,1.0,0.0,6.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60034809,FOODS_3_827_WI_3_evaluation,d_1969,NaN,1.0,5.0,2.0,2.0,0.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
# add quantile

In [114]:
for i in [28,56,168]:
    print('Rolling quantile period:', i)
    grid_df['rolling_quantile_97_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).quantile(0.97, interpolation='midpoint')).astype(np.float16)
    grid_df['rolling_quantile_87.5_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).quantile(0.875, interpolation='midpoint')).astype(np.float16)
    grid_df['rolling_quantile_50_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).quantile(0.5, interpolation='midpoint')).astype(np.float16)
    grid_df['rolling_quantile_22.5_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).quantile(0.225, interpolation='midpoint')).astype(np.float16)
    grid_df['rolling_quantile_3_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).quantile(0.03, interpolation='midpoint')).astype(np.float16)

Rolling quantile period: 28
Rolling quantile period: 56
Rolling quantile period: 168


In [115]:
grid_df.head()

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,...,rolling_quantile_97_56,rolling_quantile_87.5_56,rolling_quantile_50_56,rolling_quantile_22.5_56,rolling_quantile_3_56,rolling_quantile_97_168,rolling_quantile_87.5_168,rolling_quantile_50_168,rolling_quantile_22.5_168,rolling_quantile_3_168
0,HOBBIES_1_001_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,d_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
grid_df.tail()

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,...,rolling_quantile_97_56,rolling_quantile_87.5_56,rolling_quantile_50_56,rolling_quantile_22.5_56,rolling_quantile_3_56,rolling_quantile_97_168,rolling_quantile_87.5_168,rolling_quantile_50_168,rolling_quantile_22.5_168,rolling_quantile_3_168
60034805,FOODS_3_823_WI_3_evaluation,d_1969,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,2.0,1.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0
60034806,FOODS_3_824_WI_3_evaluation,d_1969,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
60034807,FOODS_3_825_WI_3_evaluation,d_1969,NaN,2.0,0.0,1.0,0.0,1.0,0.0,2.0,...,3.5,2.0,1.0,0.0,0.0,3.0,2.0,1.0,0.0,0.0
60034808,FOODS_3_826_WI_3_evaluation,d_1969,NaN,0.0,1.0,1.0,1.0,0.0,6.0,4.0,...,3.5,2.5,1.0,0.0,0.0,4.0,3.0,1.0,0.0,0.0
60034809,FOODS_3_827_WI_3_evaluation,d_1969,NaN,1.0,5.0,2.0,2.0,0.0,4.0,5.0,...,5.0,3.0,0.5,0.0,0.0,5.0,3.0,1.0,0.0,0.0


In [117]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60034810 entries, 0 to 60034809
Data columns (total 55 columns):
id                           category
d                            object
sales                        float64
sales_lag_28                 float16
sales_lag_29                 float16
sales_lag_30                 float16
sales_lag_31                 float16
sales_lag_32                 float16
sales_lag_33                 float16
sales_lag_34                 float16
sales_lag_35                 float16
sales_lag_36                 float16
sales_lag_37                 float16
sales_lag_38                 float16
sales_lag_39                 float16
sales_lag_40                 float16
sales_lag_41                 float16
sales_lag_42                 float16
rolling_mean_7               float16
rolling_std_7                float16
rolling_mean_14              float16
rolling_std_14               float16
rolling_mean_28              float16
rolling_std_28               float

In [118]:
grid_df['d']=grid_df['d'].apply(lambda x:int(x.split('_')[1]))
grid_df_1 = pd.read_pickle('grid_part_1.pkl')
grid_df_1=grid_df_1[['id','d']]

In [119]:
grid_df_1.head()

,id,d
0,HOBBIES_1_008_CA_1_evaluation,1
1,HOBBIES_1_009_CA_1_evaluation,1
2,HOBBIES_1_010_CA_1_evaluation,1
3,HOBBIES_1_012_CA_1_evaluation,1
4,HOBBIES_1_015_CA_1_evaluation,1


In [120]:
grid_df=grid_df_1.merge(grid_df, on=['id','d'], how='left' )

In [121]:
grid_df.head()

,id,d,sales,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,...,rolling_quantile_97_56,rolling_quantile_87.5_56,rolling_quantile_50_56,rolling_quantile_22.5_56,rolling_quantile_3_56,rolling_quantile_97_168,rolling_quantile_87.5_168,rolling_quantile_50_168,rolling_quantile_22.5_168,rolling_quantile_3_168
0,HOBBIES_1_008_CA_1_evaluation,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_009_CA_1_evaluation,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_010_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_012_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_015_CA_1_evaluation,1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
grid_df.shape

(47617813, 55)

In [124]:
print('Save lags and rollings')
grid_df.to_pickle('lags_df_'+str(SHIFT_DAY)+'_v3.pkl')

Save lags and rollings
